# Map of NZ using Geodetic Markers

In this project, we are going to use a geodetic markers to build a map of New Zealand.

First, What are "Geodetic makers"?

To answer, we need a short story.

In December of 2020, I took a small family holiday to [Mt. Maunganui](https://www.newzealand.com/int/mount-maunganui/). Of the many activiies or lack there of, we devided to climb [Mauao](https://en.wikipedia.org/wiki/Mount_Maunganui_(mountain))... twice.

At the summit, you are welcomed by a geodetic marker. And also amazing views of the township and beach.

Here, I thought of an idea to map all the markers in New Zealand and colour them based on elevation.

### Learning Points

Before we get stuck in, we will have a look at some learning points, which this post will provide.
* How to use pandas to load .csv data
* How to vary alpha in matplotlib using cmap
* pandas groupby method and aggregation

### How to access the data

To get a hold of the data we are going to use the [Koordinates website](https://koordinates.com/). This company is responsible for housing most the geospatial data for New Zealand. We will require to create an account to access this data but will is completely free.

We can search for [NZ Geodetic Vertical Marks](https://koordinates.com/from/data.linz.govt.nz/layer/50784/). This data is curated by the members of [Land Information New Zealand website](https://www.linz.govt.nz/). Selecting *Vertical Marks* gives us heights in a particular format.

When it comes to heights, a simple measurement isn't that simple.

Without getting into too much detail, vertical datums resemble what is known as normal-orthometric heights. This is what we are most familiar with as height above sea level. Normal-orthometric height is actually the mean/average heights at sea level (as sea level can change with tide).

Ellipsoidal heights is another form of height, which approximates Earth as a ellipsoid and uses this as a point of reference.

[The LINZ website goes into this with more detail](https://www.linz.govt.nz/data/geodetic-system/datums-projections-and-heights/vertical-datums).

### Import libraries and load data

In [ ]:
import pandas as pd
import numpy as np

import random as r

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import matplotlib.colors as colors
import matplotlib.cm as cmx
import seaborn as sns
%matplotlib inline

In [ ]:
# find the name of the .csv file containing the data
!ls ./files

In [ ]:
# load into dataframe
geomarks = pd.read_csv('files/nz-geodetic-vertical-marks.csv')
geomarks

In [ ]:
# explore the data more
geomarks.info()

### Exploring the Data

We can see there are a total of 105,582 entries.

The important features (input variables) are `shape_X`, `shape_Y`, which resemble longitude and latitude, and `normal_orthometric_height`, which is the height. These are all accounted for with no missing values.

Using `shape_X` and `shape_Y`, we can actually plot this out to represent geographic space. This has probably done countless times before but when I found this out in Aurélien Géron's book *Hands On Machine Learning, 2nd edition*, it blew my mind!

In [ ]:
mapNZ = geomarks.plot(kind='scatter', x='shape_X', y='shape_Y',  figsize=(15,15))
ax = plt.axis('off')

Look familiar? It's New Zealand. 

There are so many markers 105,582 to be exact. So, let's reduce the size of the markers. We can alter parameters:
* `alpha` - this is the opacity of the marker
* `s` - size of the marker

In [ ]:
mapNZ = geomarks.plot(kind='scatter', x='shape_X', y='shape_Y', 
                       figsize=(15,15), alpha=1.0, s=0.1)
ax = plt.axis('off')

Now we can apply a color based on vertical height.

You can see what some of the important parameters such as:
* `c` - this is colours of the points
* `cmap` - this provides the colour scheme; we will be using 'jet'


In [ ]:
mapNZ = geomarks.plot(kind='scatter', x='shape_X', y='shape_Y', 
                            figsize=(15,15), s=geomarks['normal_orthometric_height']/100,
                            c='normal_orthometric_height', cmap=plt.get_cmap('jet'), colorbar=True)

ax = plt.axis('off')

You may notice that the lower points are drowning out the higher ones despite varying the marker size by vertical height.

We can further see this by exploring the vertical heights. Plotting them out, we see a majority at the lower end as you would expect. The data is considered tail heavy.

In [ ]:
heights = geomarks['normal_orthometric_height']
heights.describe()

In [ ]:
height_graphs = heights.plot(kind='hist', bins=50, figsize=(15,5), title='Histogram of Heights')

What we can then do is vary the alpha through the cmap. We take the cmap and change the alpha with the colder colours being associated with lower vertical heights also having lower alpha compared to warmer colour with greater heights.

In [ ]:
x = geomarks['shape_X']
y = geomarks['shape_Y']
c = geomarks['normal_orthometric_height']
s = geomarks['normal_orthometric_height']/50

cmap = plt.cm.jet
my_cmap = cmap(np.arange(cmap.N))
my_cmap[:, -1] = np.linspace(0, 1, cmap.N)

my_cmap = ListedColormap(my_cmap)

plt.figure(figsize=(15,15))
plt.scatter(x, y, c=c, s=s, cmap=my_cmap)
ax = plt.axis('off')
cbar = plt.colorbar()

And here we have our map that we intended to create.

But let's not stop there! Let's look at other aspects of the data.

### Further Data Exploration

Let's look markers per district. Then, we can plot this data onto a bar chart.

In [ ]:
geomarks['land_district'] = geomarks['land_district'].astype('str')
by_district = geomarks['land_district'].value_counts()
by_district

We can see there are actually some `nan`, which are hidden only if we convert the column type to a string. So we need to drop these.

In [ ]:
geomarks = geomarks[geomarks['land_district']!='nan']
by_district = geomarks['land_district'].value_counts()
by_district

Here, the `nan` has been removed

In [ ]:
by_district.plot(kind='bar', title='Number of Marks by district', figsize=(7,7))
plt.show()

Next, let's look at the average heights of each district. We do this using `grouby` and `agg` or aggregation. We will also drop extra columns to aid readability

In [ ]:
district_group = geomarks.groupby('land_district') # groups data by district
district_means = district_group.agg(['mean', 'sem']) # 'sem' means standard error sem = std / n ** (1/2)
district_means = district_means.drop(['id', 'nod_id', 'shape_X', 'shape_Y'], axis=1) # dropping extra columns for readability
district_means = district_means.sort_values(by=('normal_orthometric_height', 'mean'), ascending=False)
district_means

In [ ]:
x = district_means[('normal_orthometric_height', 'mean')]
err = district_means[('normal_orthometric_height', 'sem')]

plt.figure(figsize=(15,15))

plt.barh(y=x.index, width=x.values, xerr=err, capsize=5)

plt.title('Mean Heights per District')
plt.xlabel('Mean of Height')
plt.ylabel('District')
plt.show()

We can see that Otago on average as has the highest markers.

Let's go back to the New Zealand map. What we are going to do different is that we are going to plot points but use categorical colours. This will be based on district.

In [ ]:
x = geomarks['shape_X']
y = geomarks['shape_Y']
s = geomarks['normal_orthometric_height'] / 200

plt.figure(figsize=(15,15))

colour_labels = list(district_means.index) # turns districts into a list
rgb_values = sns.color_palette('rocket', len(colour_labels)) # use seaborn to create colours
colour_map = dict(zip(colour_labels, rgb_values)) # create a dict of district as keys and colours as values

NZ_d_map = plt.scatter(x, y, alpha=1.0, s=s, c=geomarks['land_district'].map(colour_map), label=colour_labels)
plt.legend()
ax = plt.axis('off')

## References

1. Mount Maunganui - Things to see and do - North Island | New Zealand. (n.d.). Www.Newzealand.com. https://www.newzealand.com/int/mount-maunganui/
2. Mount Maunganui (mountain). (2020, November 20). Wikipedia. https://en.wikipedia.org/wiki/Mount_Maunganui_(mountain)
3. Koordinates — Earth’s Data Platform. (n.d.). Koordinates.com. https://koordinates.com/
4. NZ Geodetic Vertical Marks. (n.d.). Koordinates.com. Retrieved December 11, 2020, from https://koordinates.com/from/data.linz.govt.nz/layer/50784/
5. Land Information New Zealand (LINZ). (n.d.). Land Information New Zealand (LINZ). https://www.linz.govt.nz/
6. Vertical datums. (n.d.). Land Information New Zealand (LINZ). Retrieved December 11, 2020, from https://www.linz.govt.nz/data/geodetic-system/datums-projections-and-heights/vertical-datums
7. (2019). Chapter 2: End-to-End Machine Learning Project [Review of Chapter 2: End-to-End Machine Learning Project]. In Hands on Machine Learning (pp. 35–84). O’Reily.
8. Python R. Plot With Pandas: Python Data Visualization for Beginners – Real Python. realpython.com. Accessed December 11, 2020. https://realpython.com/pandas-plot-python/
9. How to Normalize a Pandas DataFrame Column. CodeSpeedy. Published February 19, 2020. Accessed December 11, 2020. https://www.codespeedy.com/normalize-a-pandas-dataframe-column/
10. Abdishakur. (2020, April 29). How to make Value-By-Alpha Maps in Python. Retrieved from Medium website: https://towardsdatascience.com/how-to-make-value-by-alpha-maps-in-python-484722160490
11. Python Data Analysis Library — pandas: Python Data Analysis Library. Pydata.org. Published 2018. https://pandas.pydata.org/
12. Python Pandas - GroupBy - Tutorialspoint. www.tutorialspoint.com. Accessed December 14, 2020. https://www.tutorialspoint.com/python_pandas/python_pandas_groupby.htm
13. Matplotlib Scatter Plot Color by Category in Python. kanoki. Published August 30, 2020. Accessed December 15, 2020. https://kanoki.org/2020/08/30/matplotlib-scatter-plot-color-by-category-in-python/